In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
data_file_train = r'E:\Python\P1 Data\Consumer_Complaints_train.csv'

In [3]:
cc_all = pd.read_csv(data_file_train)

In [4]:
cc_all.shape

(478421, 18)

In [5]:
cc = cc_all[cc_all['Company']=='Bank of America']

In [6]:
cc.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
4,2014-11-20,Credit card,NaN,Transaction issue,NaN,NaN,NaN,Bank of America,MA,02127,NaN,NaN,Web,2014-11-28,Closed with explanation,Yes,No,1132572
5,2014-06-26,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,Bank of America,WI,54313,NaN,NaN,Referral,2014-07-01,Closed with explanation,Yes,No,913769
9,2016-03-30,Bank account or service,Checking account,Deposits and withdrawals,NaN,I put my settlement check in to bank of americ...,Company has responded to the consumer and the ...,Bank of America,TX,787XX,NaN,Consent provided,Web,2016-04-01,Closed with explanation,Yes,No,1855737
16,2013-12-13,Mortgage,Conventional adjustable mortgage (ARM),"Loan modification,collection,foreclosure",NaN,NaN,NaN,Bank of America,FL,33134,NaN,NaN,Web,2013-12-13,Closed with explanation,Yes,No,633242
47,2012-05-19,Bank account or service,Checking account,"Account opening, closing, or management",NaN,NaN,NaN,Bank of America,FL,33312,NaN,NaN,Referral,2012-06-03,Closed with monetary relief,Yes,No,84849


In [7]:
cc.shape

(46336, 18)

In [8]:
from sklearn.model_selection import train_test_split
cc_train, cc_test = train_test_split(cc, test_size=0.2, random_state=2)

cc_train.reset_index(drop=True, inplace=True)
cc_test.reset_index(drop=True, inplace=True)

In [9]:
for col in cc_train.select_dtypes(['object']).columns:
    print(col,':',cc_train[col].nunique())

Date received : 1730
Product : 11
Sub-product : 43
Issue : 85
Sub-issue : 46
Consumer complaint narrative : 2881
Company public response : 4
Company : 1
State : 60
ZIP code : 11015
Tags : 3
Consumer consent provided? : 3
Submitted via : 6
Date sent to company : 1578
Company response to consumer : 6
Timely response? : 2
Consumer disputed? : 2


In [10]:
cc.isnull().sum()

Date received                       0
Product                             0
Sub-product                      6155
Issue                               0
Sub-issue                       45190
Consumer complaint narrative    42733
Company public response         37105
Company                             0
State                             475
ZIP code                          475
Tags                            40712
Consumer consent provided?      40200
Submitted via                       0
Date sent to company                0
Company response to consumer        0
Timely response?                    0
Consumer disputed?                  0
Complaint ID                        0
dtype: int64

In [11]:
for col in cc_test.select_dtypes(['object']).columns:
    print(col,':',cc_test[col].nunique())

Date received : 1558
Product : 11
Sub-product : 36
Issue : 73
Sub-issue : 30
Consumer complaint narrative : 718
Company public response : 2
Company : 1
State : 57
ZIP code : 5407
Tags : 3
Consumer consent provided? : 3
Submitted via : 6
Date sent to company : 1396
Company response to consumer : 6
Timely response? : 2
Consumer disputed? : 2


In [12]:
cc_train.drop(['Company','ZIP code', 'Complaint ID'], axis=1, inplace=True)
cc_test.drop(['Company','ZIP code', 'Complaint ID'], axis=1, inplace=True)

In [13]:
for col in ['Date received','Date sent to company']:
    cc_train[col]=pd.to_datetime(cc_train[col],infer_datetime_format=True)
    cc_test[col]=pd.to_datetime(cc_test[col],infer_datetime_format=True)

In [14]:
cc_train['Received Sent Date Diff']=pd.to_numeric(cc_train['Date sent to company']-cc_train['Date received'])
cc_test['Received Sent Date Diff']=pd.to_numeric(cc_test['Date sent to company']-cc_test['Date received'])

In [15]:
cc_train['Date received year'] = pd.DatetimeIndex(cc_train['Date received']).year
cc_train['Date received month'] = pd.DatetimeIndex(cc_train['Date received']).month
cc_train['Date received weekday'] = pd.DatetimeIndex(cc_train['Date received']).weekday

cc_test['Date received year'] = pd.DatetimeIndex(cc_test['Date received']).year
cc_test['Date received month'] = pd.DatetimeIndex(cc_test['Date received']).month
cc_test['Date received weekday'] = pd.DatetimeIndex(cc_test['Date received']).weekday

In [16]:
for col in ['Date received','Date sent to company']:
    cc_train.drop([col],1,inplace=True)
    cc_test.drop([col],1,inplace=True)

In [17]:
for i in range(len(cc_train)):
    if cc_train['Product'][i] in ['Mortgage']:
        cc_train.loc[i, 'Product'] = 'Mortgage'
    if cc_train['Product'][i] in ['Debt collection']:
        cc_train.loc[i, 'Product'] = 'Debt collection'
    if cc_train['Product'][i] in ['Credit reporting']:
        cc_train.loc[i, 'Product'] = 'Credit reporting'
    if cc_train['Product'][i] in ['Credit card', 'Prepaid card']:
        cc_train.loc[i, 'Product'] = 'Credit and Prepaid Card'
    if cc_train['Product'][i] in ['Bank account or service', 'Money transfers']:
        cc_train.loc[i, 'Product'] = 'Bank account or service'
    if cc_train['Product'][i] in ['Consumer Loan', 'Student loan', 'Payday loan']:
        cc_train.loc[i, 'Product'] = 'Loan services'
    if cc_train['Product'][i] in ['Other financial service']:
        cc_train.loc[i, 'Product'] = 'Other financial service'

In [18]:
for i in range(len(cc_test)):
    if cc_test['Product'][i] in ['Mortgage']:
        cc_test.loc[i, 'Product'] = 'Mortgage'
    if cc_test['Product'][i] in ['Debt collection']:
        cc_test.loc[i, 'Product'] = 'Debt collection'
    if cc_test['Product'][i] in ['Credit reporting']:
        cc_test.loc[i, 'Product'] = 'Credit reporting'
    if cc_test['Product'][i] in ['Credit card', 'Prepaid card']:
        cc_test.loc[i, 'Product'] = 'Credit and Prepaid Card'
    if cc_test['Product'][i] in ['Bank account or service', 'Money transfers']:
        cc_test.loc[i, 'Product'] = 'Bank account or service'
    if cc_test['Product'][i] in ['Consumer Loan', 'Student loan', 'Payday loan']:
        cc_test.loc[i, 'Product'] = 'Loan services'
    if cc_test['Product'][i] in ['Other financial service']:
        cc_test.loc[i, 'Product'] = 'Other financial service'

In [19]:
k=cc_train['Issue'].value_counts()
for val in k.axes[0][0:15]:
    varname='Issue_'+val.replace(',','_').replace(' ','_')
    cc_train[varname]=np.where(cc_train['Issue']==val,1,0)
    cc_test[varname]=np.where(cc_test['Issue']==val,1,0)
del cc_train['Issue']
del cc_test['Issue']

In [20]:
del cc_train['Company public response']
del cc_test['Company public response']

In [21]:
cc_train['State'].dropna(axis=0, inplace=True)
cc_test['State'].dropna(axis=0, inplace=True)

In [22]:
North_east = ['NJ', 'NY', 'PA', 'NH', 'MA', 'CT', 'ME', 'VT', 'RI', 'AE']
South = ['TX', 'OK', 'VA', 'FL', 'GA', 'NC', 'LA', 'SC', 'KY', 'MD', 'DC', 'TN', 'AL', 'DE', 
         'MS', 'AR', 'WV', 'AS', 'PR', 'MH', 'GU', 'MP', 'FM', 'VI']
Mid_West = ['IL', 'IN', 'NE', 'WI', 'MN', 'OH', 'MO', 'MI', 'SD', 'KS', 'IA', 'ND']
West = ['CA', 'OR', 'WA', 'NV', 'AZ', 'NM', 'UT', 'CO', 'ID', 'AK', 'WY', 'HI', 'MT', 'AP']

In [23]:
for i in range(len(cc_train)):
    if cc_train['State'][i] in North_east:
        cc_train.loc[i, 'State'] = 'North_East'
    if cc_train['State'][i] in South:
        cc_train.loc[i, 'State'] = 'South'
    if cc_train['State'][i] in Mid_West:
        cc_train.loc[i, 'State'] = 'Mid_West'
    if cc_train['State'][i] in West:
        cc_train.loc[i, 'State'] = 'West'

In [24]:
for i in range(len(cc_test)):
    if cc_test['State'][i] in North_east:
        cc_test.loc[i, 'State'] = 'North_East'
    if cc_test['State'][i] in South:
        cc_test.loc[i, 'State'] = 'South'
    if cc_test['State'][i] in Mid_West:
        cc_test.loc[i, 'State'] = 'Mid_West'
    if cc_test['State'][i] in West:
        cc_test.loc[i, 'State'] = 'West'

In [25]:
for col in ['Product','Submitted via','Company response to consumer','Timely response?', 'State']:
    
    temp=pd.get_dummies(cc_train[col],prefix=col,drop_first=True)
    cc_train=pd.concat([temp,cc_train],1)
    cc_train.drop([col],1,inplace=True)
    
    temp=pd.get_dummies(cc_test[col],prefix=col,drop_first=True)
    cc_test=pd.concat([temp,cc_test],1)
    cc_test.drop([col],1,inplace=True)

In [26]:
for col in ['Sub-product','Sub-issue','Consumer complaint narrative',
            'Tags','Consumer consent provided?']:
    varname=col.replace('-','_').replace('?','').replace(" ",'_')
    cc_train[varname]=np.where(pd.isnull(cc_train[col]),0,1)
    cc_train.drop([col],1,inplace=True)
    cc_test[varname]=np.where(pd.isnull(cc_test[col]),0,1)
    cc_test.drop([col],1,inplace=True)

In [27]:
cc_train['Consumer disputed?']=np.where(cc_train['Consumer disputed?']=="Yes",1,0)
cc_test['Consumer disputed?']=np.where(cc_test['Consumer disputed?']=="Yes",1,0)

In [28]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [29]:
cols = ['Received Sent Date Diff', 'Date received year', 'Date received month', 'Date received weekday']

for col in cols:
    scale = scaler.fit(cc_train[[col]])
    cc_train[col] = scale.transform(cc_train[[col]])
    cc_test[col] = scale.transform(cc_test[[col]])

In [30]:
cc_train.head()

,State_North_East,State_South,State_West,Timely response?_Yes,Company response to consumer_Closed with explanation,Company response to consumer_Closed with monetary relief,Company response to consumer_Closed with non-monetary relief,Company response to consumer_Closed with relief,Company response to consumer_Closed without relief,Submitted via_Fax,...,Issue_Making/receiving_payments__sending_money,Issue_Using_a_debit_or_ATM_card,Issue_Credit_decision_/_Underwriting,Issue_Closing/Cancelling_account,Issue_Identity_theft_/_Fraud_/_Embezzlement,Issue_Cont'd_attempts_collect_debt_not_owed,Sub_product,Sub_issue,Consumer_complaint_narrative,Consumer_consent_provided
0,0,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
2,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,1,1,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0


In [31]:
from xgboost.sklearn import XGBClassifier

In [33]:
xgb_model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0.4, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=5,
              min_child_weight=1, n_estimator=700, n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [39]:
x_train = cc_train.drop('Consumer disputed?',1)
y_train = cc_train['Consumer disputed?']

x_test = cc_test.drop('Consumer disputed?',1)
y_test = cc_test['Consumer disputed?']

In [40]:
xgb_model.fit(x_train, y_train)

[16:29:47] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0.4, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimator=700, n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [41]:
from sklearn.metrics import roc_auc_score

In [42]:
roc_auc_score(y_train, xgb_model.predict_proba(x_train)[:,1])

0.637039977748638

In [43]:
roc_auc_score(y_test, xgb_model.predict_proba(x_test)[:,1])

0.6119110192981327

In [44]:
xgb_model.predict_proba(cc_test.head(8).drop('Consumer disputed?',1))

array([[0.7874745 , 0.21252547],
       [0.7774048 , 0.22259519],
       [0.7739413 , 0.22605875],
       [0.8102412 , 0.18975876],
       [0.79084027, 0.20915976],
       [0.7751255 , 0.22487448],
       [0.7300411 , 0.2699589 ],
       [0.7870208 , 0.21297917]], dtype=float32)

In [45]:
cc_test.head(8)['Consumer disputed?']

0    1
1    0
2    0
3    0
4    0
5    0
6    0
7    1
Name: Consumer disputed?, dtype: int32

In [46]:
import pickle
# open a file, where you ant to store the data
file = open('DisputePred_xgb.pkl', 'wb')

# dump information to that file
pickle.dump(xgb_model, file)

In [47]:
cc_test.head(1)

,State_North_East,State_South,State_West,Timely response?_Yes,Company response to consumer_Closed with explanation,Company response to consumer_Closed with monetary relief,Company response to consumer_Closed with non-monetary relief,Company response to consumer_Closed with relief,Company response to consumer_Closed without relief,Submitted via_Fax,...,Issue_Making/receiving_payments__sending_money,Issue_Using_a_debit_or_ATM_card,Issue_Credit_decision_/_Underwriting,Issue_Closing/Cancelling_account,Issue_Identity_theft_/_Fraud_/_Embezzlement,Issue_Cont'd_attempts_collect_debt_not_owed,Sub_product,Sub_issue,Consumer_complaint_narrative,Consumer_consent_provided
0,1,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
